In [1]:
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
driver= webdriver.Chrome(service=ChromiumService(ChromeDriverManager().install()))
#driver.get("https://www.olx.in/en-in/items/q-cars?isSearchCall=true")
#driver.maximize_window()

In [3]:
driver.get("https://www.olx.in/en-in/items/q-cars?isSearchCall=true")
driver.maximize_window()

In [4]:
driver.implicitly_wait(10)

In [5]:
name=[]   
Distance=[]
price=[]

In [6]:
# Scroll to the element using ActionChains
#actions = ActionChains(driver)
#actions.move_to_element(element).perform()

In [7]:
brands = ['Maruti','Mahindra','Renault','Tata','Ford','Hyundai','Honda']

In [8]:
for brand in brands:
    driver.get(f"https://www.olx.in/en-in/items/q-{brand}?isSearchCall=true")
    driver.maximize_window()
    page_source = driver.page_source
    if "Oops... we didn't find anything that matches this search" in page_source:
        driver.get(f"https://www.olx.in/en-in/items/q-{brand}?isSearchCall=true")
    for i in range(0,7):
        # Scroll to the element using ActionChains
        time.sleep(3)
        element= driver.find_element(By.XPATH, "//button[@data-aut-id='btnLoadMore']")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()      
        driver.find_element(By.XPATH, "//button[@data-aut-id='btnLoadMore']").click()
    print(brand)
    time.sleep(5)
    cars=driver.find_elements(By.XPATH,"//li[@data-aut-id='itemBox']")
    names=driver.find_elements(By.XPATH, "//*[contains(@data-aut-id, 'itemTitle')]")
    try:
        driver.find_element(By.XPATH, "//*[contains(@data-aut-id, 'itemSubTitle')]").is_displayed()
        distances=driver.find_elements(By.XPATH, "//*[contains(@data-aut-id, 'itemSubTitle')]")
    except:
        distances=driver.find_elements(By.XPATH, "//*[contains(@data-aut-id, 'itemDetails')]")
    prices=driver.find_elements(By.XPATH, "//*[contains(@data-aut-id, 'itemPrice')]")
    print(len(cars))
    time.sleep(5)
    for i in range(0,len(cars)):
        
        try:
            name.append(names[i].text)      
        except:
            name.append('error')
        try:
            Distance.append(distances[i].text)
        except:
            Distance.append('error')
        try:  
            price.append(prices[i].text)    
        except:
            price.append('error')
        

Maruti
160
Mahindra
160
Renault
160
Tata
160
Ford
160
Hyundai
160
Honda
160


In [41]:
caz={"name":name,"Distance":Distance,"price":price}
df1=pd.DataFrame(caz)
df1

,name,Distance,price
0,Maruti Suzuki Alto,"2008 - 80,000 km","₹ 1,30,000"
1,Maruti Suzuki Ciaz,"2018 - 82,000 km","₹ 8,25,000"
2,Maruti Suzuki Alto 800,2017 - 34000.0 km,"₹ 3,00,000"
3,Maruti Suzuki Vitara-Brezza,"2018 - 43,000 km","₹ 8,51,000"
4,Maruti Suzuki Ertiga,"2016 - 74,000 km","₹ 5,90,000"
...,...,...,...
1115,Open land for farm houses in a peaceful surrou...,error,error
1116,"Honda Amaze 2013-2016 S i-Dtech, 2013, Diesel",error,error
1117,"Honda Amaze 1.2 EMT I VTEC, 2017, Petrol",error,error
1118,Activa 6 G,error,error


In [42]:
df1=df1[~df1['Distance'].str.contains('Monthly')]
df1=df1[~df1['Distance'].str.contains('Weekly')]
df1=df1[~df1['Distance'].str.contains('Daily')]

In [43]:
df1=df1[~df1['Distance'].str.contains('Bds')]


In [44]:
df1=df1[df1['Distance'] != 'error']

In [45]:
df1[['Brand_name','Model']]=df1['name'].str.split(' ',1,expand=True)

C:\Users\PRANESH\AppData\Local\Temp\ipykernel_12936\1806821841.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df1[['Brand_name','Model']]=df1['name'].str.split(' ',1,expand=True)


In [46]:
df1[['Purchase_year','Kms_Driven']]=df1['Distance'].str.split('-',1,expand=True)

C:\Users\PRANESH\AppData\Local\Temp\ipykernel_12936\2810068760.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df1[['Purchase_year','Kms_Driven']]=df1['Distance'].str.split('-',1,expand=True)


In [47]:
df1.drop(['name'],axis=1,inplace=True)

In [48]:
df1.drop(['Distance'],axis=1,inplace=True)

In [49]:
df1=df1[['Brand_name','Model','Purchase_year','Kms_Driven','price']]

In [50]:
df=df1[df1['Purchase_year'] !='error']

In [51]:
df['Kms_Driven'] = df['Kms_Driven'].str.replace('km','')
df['price'] = df['price'].str.replace('₹','')
df['price'] = df['price'].str.replace(' ','')
df['price'] = df['price'].str.replace(',','')
df['Kms_Driven'] = df['Kms_Driven'].str.replace(' ','')
df['Kms_Driven'] = df['Kms_Driven'].str.replace(',','')

In [55]:
df['Kms_Driven'].astype('float')
df['Purchase_year'].astype('int')
df['price'].astype('int')

0       80000.0
1       82000.0
2       34000.0
3       43000.0
4       74000.0
         ...   
1095    22000.0
1096    78000.0
1097    72000.0
1098     3461.0
1099    73000.0
Name: Kms_Driven, Length: 1068, dtype: float64

In [39]:
df

,Brand_name,Model,Purchase_year,Kms_Driven,price
0,Maruti,Suzuki Alto,2008,80000,130000
1,Maruti,Suzuki Ciaz,2018,82000,825000
2,Maruti,Suzuki Alto 800,2017,34000.0,300000
3,Maruti,Suzuki Vitara-Brezza,2018,43000,851000
4,Maruti,Suzuki Ertiga,2016,74000,590000
...,...,...,...,...,...
1095,Honda,"City ZX, 2022, Diesel",2022,22000,529999
1096,Honda,"WR-V 1.5 VX i-DTEC, 2019, Diesel",2013,78000,790000
1097,Mercedes-Benz,"A Class 2013-2015 A180 CDI, 2014, Diesel",2017,72000,190000
1098,Honda,"Jazz 1.5 V i DTEC, 2016, Diesel",2022,3461,925000


In [40]:
df.to_excel("cars_1.xlsx")